In [1]:
import pandas as pd
import numpy as np

In [2]:
matches = pd.read_csv("../../../preprocessing/data/matches.csv")
matches = matches.sort_values(by='Date')

In [3]:
matches['tournament_level'].unique()

array(['ATP250', 'Grand Slam', 'ATP500', 'Masters 1000'], dtype=object)

## We'll use Brier score to evaluate performance of ELO ratings. First we'll calculate Brier score when using rank difference to calculate expected probabilities as a baseline.

In [4]:
def logistic_function(x):
    return 1 / (1 + np.exp(-x))


class ATPRankingPredictions:
    def __init__(self, matches_df):
        self.matches_df = matches_df

    def calculate_winner_probabilities(self):
        expected_probabilities = []
        
        for index, row in self.matches_df.iterrows():
            winner_rank = row['winner_rank']
            loser_rank = row['loser_rank']
            
            if pd.isna(winner_rank):
                winner_rank = 2000
            if pd.isna(loser_rank):
                loser_rank = 2000
                 
            rank_difference = loser_rank - winner_rank
            
            scaled_rank_difference = rank_difference / 100.0  # Dla 100 jest najlepszy wynik
            expected_winner = logistic_function(scaled_rank_difference)
            
            expected_probabilities.append(expected_winner)
        
        return expected_probabilities
    
    def calculate_prediction_accuracy(self):
        correct_predictions = 0 
        valid_count = 0
        for index, row in self.matches_df.iterrows():
            winner_rank = row['winner_rank']
            loser_rank = row['loser_rank']
            valid_count += 1
            if winner_rank < loser_rank:
                correct_predictions+=1
                
        return (correct_predictions/valid_count) * 100
    
    def calculate_brier_score(self):
        total_brier_score = 0
        
        for expected_winner_probability in self.calculate_winner_probabilities():
            
            total_brier_score += (1 - expected_winner_probability) ** 2
        
        return total_brier_score / len(self.matches_df)

In [5]:
atp_rank_predictor = ATPRankingPredictions(matches)

prediction_accuracy = atp_rank_predictor.calculate_prediction_accuracy()
print(f"Prediction accuracy: {prediction_accuracy}")

brier_score_rank = atp_rank_predictor.calculate_brier_score()
print(f"Brier Score (Rank-based Prediction with Logistic Normalization): {brier_score_rank}")

Prediction accuracy: 63.04301608674084
Brier Score (Rank-based Prediction with Logistic Normalization): 0.2323637225456384


In [6]:
def create_player_ranks(matches_df):
    player_ranks = {}

    for index, row in matches_df.iterrows():
        winner_id = row['winner_id']
        loser_id = row['loser_id']
        winner_rank = row['winner_rank']
        loser_rank = row['loser_rank']
        
        if winner_id not in player_ranks:
            player_ranks[winner_id] = winner_rank
        
        if loser_id not in player_ranks:
            player_ranks[loser_id] = loser_rank

    return player_ranks


In [7]:
player_ranks = create_player_ranks(matches)

### Considering only matches from last year for match count

In [7]:
class EloRatingsPredictor:
    def __init__(self, matches_df):
        self.matches_df = matches_df
        self.current_ratings = self.initialize_ratings()
    
    def initialize_ratings(self):
        players = set(self.matches_df['winner_id']).union(set(self.matches_df['loser_id']))
        ratings = {player: 1500 for player in players}
        return ratings
    
    @staticmethod
    def k_factor(matches_history, player_id):
        total_matches = len(matches_history[player_id])
        return 250 / ((total_matches + 5) ** 0.4)
    
    @staticmethod
    def win_probability(rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    @staticmethod
    def update_matches_history(matches_history, player_id, current_date):
        if isinstance(current_date, str):
            current_date = datetime.strptime(current_date, "%Y-%m-%d")
        
        one_year_ago = current_date - timedelta(days=365)
    
        matches_history[player_id] = [date for date in matches_history[player_id] if date > one_year_ago]
    
        matches_history[player_id].append(current_date)

    
    def update_ratings(self, tourney_level, matches_history, winner_id, loser_id, current_date):
        rating_winner = self.current_ratings[winner_id]
        rating_loser = self.current_ratings[loser_id]
                
        winner_probability = self.win_probability(rating_winner, rating_loser)
        loser_probability = 1 - winner_probability
        
        self.update_matches_history(matches_history, winner_id, current_date)
        self.update_matches_history(matches_history, loser_id, current_date)
        
        k_winner = self.k_factor(matches_history, winner_id)
        k_loser = self.k_factor(matches_history, loser_id)
        k_level = 1.1 if tourney_level == "Grand Slam" else 1
        
        self.current_ratings[winner_id] = rating_winner + (k_winner * k_level) * (1 - winner_probability)
        self.current_ratings[loser_id] = rating_loser + (k_loser * k_level) * (0 - loser_probability)
        return rating_winner, rating_loser
    
    def insert_elo_ratings_to_df(self):
        matches_history = defaultdict(lambda: defaultdict(list))
            
        elo_winners = []
        elo_losers = [] 
        for index, row in self.matches_df.iterrows():
            rating_winner, rating_loser = self.update_ratings(row['tournament_level'], matches_history, row['winner_id'], row['loser_id'], row['Date'])
            elo_winners.append(rating_winner)
            elo_losers.append(rating_loser)
        
        self.matches_df['elo_winner'] = elo_winners
        self.matches_df['elo_loser'] = elo_losers
        
    def calculate_prediction_accuracy(self):
        correct_predictions = 0 
        start_score_index = 2000

        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                rating_winner = row['elo_winner']
                rating_loser = row['elo_loser']
                expected_winner = self.win_probability(rating_winner, rating_loser)
                if expected_winner > 0.5:
                    correct_predictions+=1
                
        return (correct_predictions / (len(self.matches_df) - start_score_index)) * 100
    
    def calculate_brier_score(self):
        total_brier_score = 0
        start_score_index = 2000
        
        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                rating_winner = row['elo_winner']
                rating_loser = row['elo_loser']
                expected_winner = self.win_probability(rating_winner, rating_loser)
                total_brier_score += (expected_winner - 1) ** 2
            
        return total_brier_score / (len(self.matches_df) - start_score_index)

In [8]:
elo_ratings_predictor = EloRatingsPredictor(matches)
elo_ratings_predictor.insert_elo_ratings_to_df()

elo_prediction_accuracy = elo_ratings_predictor.calculate_prediction_accuracy()
print(f"Prediction accuracy: {elo_prediction_accuracy}")
elo_brier_score = elo_ratings_predictor.calculate_brier_score()
print(f"Brier Score: {elo_brier_score}")

Prediction accuracy: 63.46296755016101
Brier Score: 0.22394325327770165


### With starting elo rating based on atp rank

In [9]:
class EloRatingsPredictor:
    def __init__(self, matches_df):
        self.matches_df = matches_df
        self.current_ratings = self.initialize_ratings()
    
    def initialize_ratings(self):
        players = set(self.matches_df['winner_id']).union(set(self.matches_df['loser_id']))
        ratings = {}

        for player in players:
            atp_rank = player_ranks.get(player, 1000)
            
            if atp_rank <= 10:
                initial_rating = 2000
            elif atp_rank <= 50:
                initial_rating = 1900
            elif atp_rank <= 100:
                initial_rating = 1800
            elif atp_rank <= 500:
                initial_rating = 1600
            else:
                initial_rating = 1500

            ratings[player] = initial_rating
        return ratings
    
    
    @staticmethod
    def k_factor(matches_count, player_id):
       return 250 / ((matches_count.get(player_id, 0) + 5) ** 0.4)
    
    @staticmethod
    def win_probability(rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    def update_ratings(self, tourney_level, matches_count, winner_id, loser_id):
        rating_winner = self.current_ratings[winner_id]
        rating_loser = self.current_ratings[loser_id]
                
        winner_probability = self.win_probability(rating_winner, rating_loser)
        loser_probability = 1 - winner_probability
        
        k_winner = self.k_factor(matches_count, winner_id)
        k_loser = self.k_factor(matches_count, loser_id)
        k_level = 1.1 if tourney_level == "Grand Slam" else 1
        
        self.current_ratings[winner_id] = rating_winner + (k_winner * k_level) * (1 - winner_probability)
        self.current_ratings[loser_id] = rating_loser + (k_loser * k_level) * (0 - loser_probability)
        return rating_winner, rating_loser
    
    def insert_elo_ratings_to_df(self):
        matches_count = defaultdict(int)
        elo_winners = []
        elo_losers = [] 
        for index, row in self.matches_df.iterrows():
            matches_count[row['winner_id']] +=1
            matches_count[row['loser_id']] +=1
            rating_winner, rating_loser = self.update_ratings(row['tournament_level'], matches_count, row['winner_id'], row['loser_id'])
            elo_winners.append(rating_winner)
            elo_losers.append(rating_loser)
        
        self.matches_df['elo_winner'] = elo_winners
        self.matches_df['elo_loser'] = elo_losers
        
    def calculate_prediction_accuracy(self):
        correct_predictions = 0 
        start_score_index = 2000

        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                rating_winner = row['elo_winner']
                rating_loser = row['elo_loser']
                expected_winner = self.win_probability(rating_winner, rating_loser)
                if expected_winner > 0.5:
                    correct_predictions+=1
                
        return (correct_predictions / (len(self.matches_df) - start_score_index)) * 100
    
    def calculate_brier_score(self):
        total_brier_score = 0
        start_score_index = 2000
        
        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                rating_winner = row['elo_winner']
                rating_loser = row['elo_loser']
                expected_winner = self.win_probability(rating_winner, rating_loser)
                total_brier_score += (expected_winner - 1) ** 2
            
        return total_brier_score / (len(self.matches_df) - start_score_index)

In [10]:
elo_ratings_predictor = EloRatingsPredictor(matches)
elo_ratings_predictor.insert_elo_ratings_to_df()

elo_prediction_accuracy = elo_ratings_predictor.calculate_prediction_accuracy()
print(f"Prediction accuracy: {elo_prediction_accuracy}")
elo_brier_score = elo_ratings_predictor.calculate_brier_score()
print(f"Brier Score: {elo_brier_score}")

Prediction accuracy: 64.0492114606556
Brier Score: 0.22241568203270798


### All additions

In [8]:
from collections import defaultdict
from datetime import datetime, timedelta

class ComplexEloRatingsPredictor:
    def __init__(self, matches_df):
        self.matches_df = matches_df
        self.current_ratings = self.initialize_ratings()
        self.momentum = defaultdict(lambda: 1.0)

    def initialize_ratings(self):
        players = set(self.matches_df['winner_id']).union(set(self.matches_df['loser_id']))
        surface_ratings = {}

        for player in players:
            atp_rank = player_ranks.get(player, 1000)
            
            if atp_rank <= 10:
                initial_rating = 2000
            elif atp_rank <= 50:
                initial_rating = 1900
            elif atp_rank <= 100:
                initial_rating = 1800
            elif atp_rank <= 500:
                initial_rating = 1600
            else:
                initial_rating = 1500

            surface_ratings[player] = {
                'General': initial_rating,
                'Hard': initial_rating,
                'Clay': initial_rating,
                'Grass': initial_rating
            }
        
        return surface_ratings

    def blended_rating(self, overall_rating, surface_rating, num_surface_matches, base_weight=0.2):
        weight = base_weight if num_surface_matches < 10 else 0.4
        return (weight * surface_rating) + ((1 - weight) * overall_rating)

    def dynamic_k_factor(self, matches_by_date, player_id):
        total_matches = len(matches_by_date[player_id])
        base_k = 210 / ((total_matches + 5) ** 0.5)
        
        k = base_k * self.momentum[player_id]
        return k

    @staticmethod
    def update_matches_history(matches_history, player_id, surface, current_date):
        if isinstance(current_date, str):
            current_date = datetime.strptime(current_date, "%Y-%m-%d")
        
        one_year_ago = current_date - timedelta(days=365)
    
        matches_history[surface][player_id] = [date for date in matches_history[surface][player_id] if date > one_year_ago]
    
        matches_history[surface][player_id].append(current_date)

    def update_momentum(self, winner_id, loser_id):
        self.momentum[winner_id] = min(self.momentum[winner_id] + 0.06, 1.3)
        
        self.momentum[loser_id] = max(self.momentum[loser_id] - 0.06, 0.7)
        
        for player in self.momentum:
            if player != winner_id and player != loser_id:
                self.momentum[player] = 1.0 + (self.momentum[player] - 1.0) * 0.99

    def win_probability(self, rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

    def update_ratings(self, tourney_level, matches_history, winner_id, loser_id, surface, current_date):
        general_rating_winner = self.current_ratings[winner_id]['General']
        general_rating_loser = self.current_ratings[loser_id]['General']
        surface_rating_winner = self.current_ratings[winner_id][surface]
        surface_rating_loser = self.current_ratings[loser_id][surface]

        general_winner_probability = self.win_probability(general_rating_winner, general_rating_loser)
        general_loser_probability = 1 - general_winner_probability
        surface_winner_probability = self.win_probability(surface_rating_winner, surface_rating_loser)
        surface_loser_probability = 1 - surface_winner_probability


        self.update_matches_history(matches_history, winner_id, 'General', current_date)
        self.update_matches_history(matches_history, loser_id, 'General', current_date)
        self.update_matches_history(matches_history, winner_id, surface, current_date)
        self.update_matches_history(matches_history, loser_id, surface, current_date)

        k_level = 1.2 if tourney_level == "Grand Slam" else (1.05 if tourney_level == "Masters" else (1.0 if tourney_level == "ATP 500" else 0.9))
        
        k_general_winner = self.dynamic_k_factor(matches_history['General'], winner_id) * k_level
        k_general_loser = self.dynamic_k_factor(matches_history['General'], loser_id) * k_level
        k_surface_winner = self.dynamic_k_factor(matches_history[surface], winner_id) * k_level
        k_surface_loser = self.dynamic_k_factor(matches_history[surface], loser_id) * k_level

        self.current_ratings[winner_id]['General'] += k_general_winner * (1 - general_winner_probability)
        self.current_ratings[loser_id]['General'] += k_general_loser * (0 - general_loser_probability)
        self.current_ratings[winner_id][surface] += k_surface_winner * (1 - surface_winner_probability)
        self.current_ratings[loser_id][surface] += k_surface_loser * (0 - surface_loser_probability)

        self.update_momentum(winner_id, loser_id)

        return general_rating_winner, general_rating_loser, surface_rating_winner, surface_rating_loser

    def insert_elo_ratings_to_df(self):
        matches_history = {
            'General': defaultdict(lambda: defaultdict(list)),
            'Hard': defaultdict(lambda: defaultdict(list)),
            'Clay': defaultdict(lambda: defaultdict(list)),
            'Grass': defaultdict(lambda: defaultdict(list))
        }
        elo_winners, elo_losers = [], []
        surface_elo_winners, surface_elo_losers = [], []
        blended_elo_winners, blended_elo_losers = [], []

        for index, row in self.matches_df.iterrows():
            winner_id, loser_id, surface, current_date = row['winner_id'], row['loser_id'], row['Surface'], row['Date']
            general_rating_winner, general_rating_loser, surface_rating_winner, surface_rating_loser = self.update_ratings(
                row['tournament_level'], matches_history, winner_id, loser_id, surface, current_date)
            
            num_surface_matches_winner = len(matches_history[surface][winner_id])
            num_surface_matches_loser = len(matches_history[surface][loser_id])
            
            elo_winners.append(general_rating_winner)
            elo_losers.append(general_rating_loser)
            surface_elo_winners.append(surface_rating_winner)
            surface_elo_losers.append(surface_rating_loser)
            blended_elo_winners.append(self.blended_rating(general_rating_winner, surface_rating_winner, num_surface_matches_winner))
            blended_elo_losers.append(self.blended_rating(general_rating_loser, surface_rating_loser, num_surface_matches_loser))
            
        self.matches_df['elo_winner'] = elo_winners
        self.matches_df['elo_loser'] = elo_losers
        self.matches_df['surface_elo_winner'] = surface_elo_winners
        self.matches_df['surface_elo_loser'] = surface_elo_losers
        self.matches_df['blended_elo_winner'] = blended_elo_winners
        self.matches_df['blended_elo_loser'] = blended_elo_losers


    def calculate_prediction_accuracy(self):
        correct_predictions = 0
        start_score_index = 2000
        valid_count = 0
        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                blended_elo_winner = row['blended_elo_winner']
                blended_elo_loser = row['blended_elo_loser']

                expected_winner = self.win_probability(blended_elo_winner, blended_elo_loser)
                valid_count += 1
                if expected_winner > 0.5:
                    correct_predictions += 1

        return (correct_predictions / valid_count) * 100

    def calculate_brier_score(self):
        total_brier_score = 0
        start_score_index = 0
        valid_count = 0
        for index, row in self.matches_df.iterrows():
            if index > start_score_index:
                blended_elo_winner = row['blended_elo_winner']
                blended_elo_loser = row['blended_elo_loser']
                expected_winner = self.win_probability(blended_elo_winner, blended_elo_loser)
                total_brier_score += (expected_winner - 1) ** 2
                valid_count +=1 
        return total_brier_score / valid_count if valid_count > 0 else float('nan')


In [9]:
complex_elo_ratings_predictor = ComplexEloRatingsPredictor(matches)
complex_elo_ratings_predictor.insert_elo_ratings_to_df()

elo_prediction_accuracy = complex_elo_ratings_predictor.calculate_prediction_accuracy()
print(f"Prediction accuracy: {elo_prediction_accuracy}")
elo_brier_score = complex_elo_ratings_predictor.calculate_brier_score()
print(f"Brier Score: {elo_brier_score}")

Prediction accuracy: 64.3682906688687
Brier Score: 0.2203066981271551


## GLICKO 2

In [13]:
import pandas as pd
from glicko2 import Player 
from math import sqrt, pi


players = {}
q = 0.0057565

def g(rd):
    return 1 / sqrt(1 + (3 * (q ** 2) * (rd ** 2)) / (pi ** 2))

def expected_outcome(player1, player2):
    g_rd = g(sqrt(player1.rd**2 + player2.rd**2))
    rating_diff = (player1.rating - player2.rating) / 400
    return 1 / (1 + 10 ** (-g_rd * rating_diff))

def get_player(player_id):
    if player_id not in players:
        players[player_id] = Player()  # Initialize new player with default Glicko-2 values
    return players[player_id]

def update_ratings(winner, loser):
    winner_rating = winner.rating
    winner_rd = winner.rd
    loser_rating = loser.rating
    loser_rd = loser.rd

    winner.update_player([loser_rating], [loser_rd], [1])
    loser.update_player([winner_rating], [winner_rd], [0])

from sklearn.metrics import precision_score, brier_score_loss

def evaluate_glicko2(matches_df):
    probabilities = []
    actuals = []
    good_pred = 0
    total_pred = 0
    for index, row in matches_df.iterrows():
        winner_id = row['winner_id']
        loser_id = row['loser_id']

        winner = get_player(winner_id)
        loser = get_player(loser_id)
        if index > 2000:
            print(winner.rating)
            print(loser.rating)
    
            win_prob = expected_outcome(winner, loser)
            print(win_prob)
            probabilities.append(win_prob)
            
            if win_prob > 0.5:
                good_pred+=1
            actuals.append(1)
            total_pred +=1

        update_ratings(winner, loser)

    accuracy = (good_pred / total_pred) * 100
    print(f"Prediction Accuracy: {accuracy:.2f}%")

    brier_score = brier_score_loss(actuals, probabilities)
    print(f"Brier Score: {brier_score:.4f}")

evaluate_glicko2(matches)

1514.5193280999658
1692.5885339308868
0.27708507405629323
1553.4637193848296
1743.868048024434
0.26482363038430035
2086.8059566109214
1638.7347784111953
0.9168004686018005
1743.754071389368
1793.7488537702334
0.43215103430273
1959.2242708394797
1545.5198004985841
0.9032344536862664
1686.4465029249457
1989.768974789486
0.1603303934241118
1959.2238399864036
1644.5501436922025
0.847532693210842
1838.8787006820448
1687.7359502142842
0.6950069417795247
1905.4977633882454
1713.9117372076591
0.7394370671905569
1848.7030638459578
1625.785786989642
0.7695397294122283
1780.9363773168234
1637.5764025616484
0.6720159312583696
1829.0970773989154
1634.550219725938
0.7423558364166881
1815.7988336590452
1608.2088802661592
0.7553579908308784
1856.3187352900786
1760.0541036745403
0.6283792550930822
2007.1062746848572
1815.040003568868
0.7379806960496239
1790.0085609024582
1713.0452992244516
0.6035322823068441
1840.4493612316214
1864.5447155129789
0.4677686971544109
1662.9022241685877
1603.7491134988863


### Zapisujemy elo all additions

In [14]:
matches[["match_id", "elo_winner", "elo_loser", "surface_elo_winner", "surface_elo_loser", "blended_elo_winner", "blended_elo_loser"]].to_csv("../../data/created_features_separate/elo.csv", index=False)